# DEEP LEARNING PROJECT 1
---

In [4]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

import matplotlib.pyplot as plt
import torch.nn as nn
from torchsummary import summary
import torch
import torch.utils.data as data

# import helpers
import ..config
from datasets import PairDataset, SingleDataset

from train.train import train_basic
from train.train import train_siamese

from models.basic_net import BasicNet
from models.oscar_net import OscarNet
from models.desmond_net import DesmondNet
from models.robert_net import RobertNet
from models.leonard_net import LeonardNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'datasets'

## PARAMETERS

In [ ]:
INPUT_SIZE = (2, 14, 14)

LEARNING_RATE = 0.00001
SUB_CRITERION = nn.CrossEntropyLoss()
FINAL_CRITERION = nn.BCELoss()
EPOCHS = 20

DEF_NET_HIDDEN_LAYER = 275
BASE_CHANNEL_SIZE = 8

HIDDEN_LAYERS = [100, 200, 300, 400, 500]
NB_HIDDENS = [1, 2, 3, 4, 5]

TRAIN_BATCH_SIZE = 1

ALPHA = 0.5
BETA = 0.25
GAMMA = 0.25
WEIGHTS_LOSS = ALPHA, BETA, GAMMA

## Load Data

In [ ]:
pairs = helpers.generate_pair_sets(config.NB_SAMPLES)

train_dataset = PairDataset(pairs[0], pairs[1], pairs[2])
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

test_dataset = PairDataset(pairs[3], pairs[4], pairs[5])
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=config.TEST_BATCH_SIZE, shuffle=True)

## Model 1: 
## Siamese/TWO FCNS

In [ ]:
models={}

for nb_outer in NB_HIDDENS:
    for hidden_outer in HIDDEN_LAYERS:
        for nb_inner in NB_HIDDENS:
            for hidden_inner in HIDDEN_LAYERS:
                inner_model = OscarNet(nb_inner, hidden_inner)
                outer_model = LeonardNet(nb_outer, oscars[-1], hidden_outer)
                
                tr_loss, tr_acc, tr_loss_l, tr_loss_r, te_loss, te_acc, te_loss_l, te_loss_r = train_siamese(model = outer_model,
                                     dataloader = train_dataloader,
                                     test_dataloader = test_dataloader,
                                     epochs = EPOCHS,
                                     final_criterion = FINAL_CRITERION, 
                                     learning_rate = LEARNING_RATE,
                                     aux_loss = True,
                                     sub_criterion = SUB_CRITERION, 
                                     weights_loss = WEIGHTS_LOSS)
                
                run_string = "{0}_{1}_{2}_{3}".format(nb_outer, hidden_outer, nb_hidden, hidden_inner)
                
                models[run_string] = {
                    'tr_loss' = tr_loss,
                    'tr_acc' = tr_acc,
                    'tr_loss_l' = tr_loss_l,
                    'tr_loss_r' = tr_loss_r,
                    'te_loss' = te_loss,
                    'te_acc' = te_acc,
                    'te_loss_l' = te_loss_l,
                    'te_loss_r' = te_loss_r
                }